## Agents & MCP

In [19]:
import random
import os
import json

from IPython.display import display, HTML
import markdown
import asyncio
from dotenv import load_dotenv
from openai import OpenAI
from fastmcp import Client

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Q1. Define function description

In [24]:
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

get_weather_tool_description = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the current weather for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city, e.g., Berlin, London, New York"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}


In [21]:
class Tools:
    def __init__(self):
        self.tools = {}
        self.functions = {}

    def add_tool(self, function, description):
        self.tools[function.__name__] = description
        self.functions[function.__name__] = function

    def get_tools(self):
        return list(self.tools.values())

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        f = self.functions[function_name]
        result = f(**arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }


def shorten(text, max_length=50):
    if len(text) <= max_length:
        return text

    return text[:max_length - 3] + "..."


class ChatInterface:
    def input(self):
        question = input("You:")
        return question

    def display(self, message):
        print(message)

    def display_function_call(self, entry, result):
        call_html = f"""
            <details>
            <summary>Function call: <tt>{entry.name}({shorten(entry.arguments)})</tt></summary>
            <div>
                <b>Call</b>
                <pre>{entry}</pre>
            </div>
            <div>
                <b>Output</b>
                <pre>{result['output']}</pre>
            </div>

            </details>
        """
        display(HTML(call_html))

    def display_response(self, entry):
        response_html = markdown.markdown(entry.content[0].text)
        html = f"""
            <div>
                <div><b>Assistant:</b></div>
                <div>{response_html}</div>
            </div>
        """
        display(HTML(html))



class ChatAssistant:
    def __init__(self, tools, developer_prompt, chat_interface, client):
        self.tools = tools
        self.developer_prompt = developer_prompt
        self.chat_interface = chat_interface
        self.client = client

    def gpt(self, chat_messages):
        return self.client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=self.tools.get_tools(),
        )


    def run(self):
        chat_messages = [
            {"role": "developer", "content": self.developer_prompt},
        ]

        # Chat loop
        while True:
            question = self.chat_interface.input()
            if question.strip().lower() == 'stop':
                self.chat_interface.display("Chat ended.")
                break

            message = {"role": "user", "content": question}
            chat_messages.append(message)

            while True:  # inner request loop
                response = self.gpt(chat_messages)

                has_messages = False

                for entry in response.output:
                    chat_messages.append(entry)

                    if entry.type == "function_call":
                        result = self.tools.function_call(entry)
                        chat_messages.append(result)
                        self.chat_interface.display_function_call(entry, result)

                    elif entry.type == "message":
                        self.chat_interface.display_response(entry)
                        has_messages = True

                if has_messages:
                    break


In [25]:
def set_weather(city: str, temp: float) -> str:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

set_weather_tool_description = {
    "type": "function",
    "name": "set_weather",
    "description": "Set or update the weather temperature for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to set the weather for, e.g., Paris, Rome"
            },
            "temp": {
                "type": "number",
                "description": "The temperature in Celsius to set for the city"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [10]:
client = OpenAI()

_tools = Tools()
_tools.add_tool(get_weather, get_weather_tool_description)
_tools.add_tool(set_weather, set_weather_tool_description)

_chat_interface = ChatInterface()

developer_prompt_1 = """
You are a sophisticated weather management assistant.
Prioritize retrieving existing weather data first when asked about a city's weather.
If a user asks to "set" or "update" weather, use the `set_weather` tool.
Be helpful and confirm once data is set.
If the city is not explicitly stated or temperature is missing for `set_weather`, ask for clarification.
"""

_assistant = ChatAssistant(
    developer_prompt=developer_prompt_1,
    chat_interface=_chat_interface,
    client=client,
    tools=_tools,
)

## Q2. Adding another tool

Now let's write a description for it.

What did you write?

In [11]:
_assistant.run()

Chat ended.


In [15]:
from fastmcp import FastMCP

mcp = FastMCP("Weather Service 🌦️")

# In a real application, known_weather_data might be a persistent database or cache
known_weather_data = {
    'berlin': 20.0
}

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    # Simulate random weather if not in known data
    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

## Q3. Run FastMCP

What's the version of FastMCP you installed?

> fastmcp==2.9.2

In [16]:
if __name__ == "__main__":
    # Check if an asyncio event loop is already running
    try:
        loop = asyncio.get_running_loop()
        if loop.is_running():
            print("Detected running asyncio loop. Running FastMCP in a new task...")
            # If a loop is already running (e.g., in Jupyter),
            # run mcp.run() as a task in the existing loop.
            # This allows the Jupyter kernel to remain responsive.
            asyncio.create_task(mcp.run_async())
        else:
            mcp.run() # Run normally if no loop is running
    except RuntimeError:
        # No running loop, so run normally
        mcp.run()

Detected running asyncio loop. Running FastMCP in a new task...


## Q4. Simple MCP Server

What do you see in the output?

Look for a string that matches this template:

Starting MCP server 'Weather Service 🌦' with transport '<TODO>'
What do you have instead of <TODO>?

> `'stdio'`

## Q5. Protocol

There are different ways to communicate with an MCP server.
Ours is currently running using standart input/output, which
means that the client write something to stdin and read the
answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

- First, we send an initialization request -- this way, we register our client with the server:
    ```json
    {"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
    ```
    We should get back something like that, which is an aknowledgement of the request:
    ```json
    {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
    ```
-  Next, we reply back, confirming the initialization:
    ```json
    {"jsonrpc": "2.0", "method": "notifications/initialized"}
    ```
    We don't expect to get anything in response
- Now we can ask for a list of available methods:
    ```json
    {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
    ```
- Let's ask the temperature in Berlin:
    ```json
    {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
    ```
- What did you get in response?

> ```json
{'jsonrpc': '2.0', 'id': 3, 'error': {'code': -32602, 'message': 'Invalid request parameters', 'data': ''}}
```

## Q6. Client

Use the client to get the list of available tools
of our script. How does the result look like?

In [31]:
async def main():
    # For Jupyter notebook - using the server instance directly
    import server  # Import your server.py file

    async with Client(server.mcp) as mcp_client:
        print("=== Getting list of available tools ===")
        tools = await mcp_client.list_tools()
        print(f"Available tools: {tools}")

        print("\n=== Testing get_weather tool ===")
        weather_result = await mcp_client.call_tool("get_weather", {"city": "Berlin"})
        print(f"Weather in Berlin: {weather_result}")


In [32]:
# Run the async function
await main()

=== Getting list of available tools ===
Available tools: [Tool(name='get_weather', description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, annotations=None), Tool(name='set_weather', description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, annotations=None)]

=== Testing get_weather tool ===
Weather in Berlin: [TextContent